<a href="https://colab.research.google.com/github/akash166d/sparkNLP/blob/master/DoD_Analysis_TestDoc_Mar21.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Package/Lib

This is required to ensure the ubuntu related dependencies are up to date

In [1]:
! sudo apt-get update --fix-missing
! sudo apt-get upgrade --fix-missing

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:11 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic/main Sources [1,745 kB]
Ign:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:13 https://developer.download.nvidia.co

In [2]:
import os

# Install java
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
! java -version

# Install pyspark
! pip install --ignore-installed -q pyspark==2.4.4

# Install Spark NLP
! pip install --ignore-installed -q spark-nlp==2.5.4

! pip install --user -U nltk

openjdk version "1.8.0_282"
OpenJDK Runtime Environment (build 1.8.0_282-8u282-b08-0ubuntu1~18.04-b08)
OpenJDK 64-Bit Server VM (build 25.282-b08, mixed mode)
     |████████████████████████████████| 215.7MB 70kB/s 
     |████████████████████████████████| 204kB 13.2MB/s 
     |████████████████████████████████| 133kB 12.8MB/s 
     |████████████████████████████████| 1.4MB 12.0MB/s 
  Created wheel for nltk: filename=nltk-3.5-cp37-none-any.whl size=1434672 sha256=c861ba19229f422be327e9e518440e091b0fc5099354b0a2f15534a617f5943e
  Stored in directory: /root/.cache/pip/wheels/ae/8c/3f/b1fe0ba04555b08b57ab52ab7f86023639a526d8bc8d384306
Successfully built nltk
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [3]:
from pyspark import SparkContext, SparkConf, SQLContext
import pyspark.sql.functions as f
from pyspark.sql.types import *
from pyspark.sql import Column, SparkSession
from pyspark.sql.window import Window
#from pyspark_llap import HiveWarehouseSession
import numpy as np
from pyspark.sql.functions import *
import pandas as pd 

from re import sub

import sparknlp
 

from pyspark.ml import Pipeline
 
from sparknlp.annotator import *
 
from sparknlp.common import *
 
from sparknlp.base import *
 
from sparknlp.pretrained import PretrainedPipeline
 
import nltk
 
from nltk.corpus import stopwords
nltk.download('stopwords')

from pyspark.sql import functions as F

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version; ", spark.version)

Spark NLP version:  2.5.4
Apache Spark version;  2.4.4


# Mount Drive and read data

In [5]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


Some of the sparkNLP annotators were having trouble reading spaces in path, hence used symbolic link to replace the path

In [6]:
! ln -s "/content/drive/My Drive" "/content/MyDrive"

In [7]:

song_data = spark.read.format("csv").option("header", "true").load("/content/drive/My Drive/SparkNLP/song_2k.csv")


In [8]:
from pyspark.sql import functions as F
song_data = song_data.select(['Key', 'Lyric', 'Genre']).withColumnRenamed('Lyric','text')
from pyspark.sql.functions import monotonically_increasing_id 

song_data = song_data.select("*").withColumn("Id", monotonically_increasing_id())
# train, test = trainDataset.randomSplit(weights=[0.5, 0.5], seed=123)
song_data = song_data.limit(10)
print(song_data.count())

song_data = song_data.filter(song_data.text != '')
print(song_data.count())
song_data = song_data.filter((song_data.Genre == 'Rock') | (song_data.Genre == 'Hip Hop') | (song_data.Genre == 'Pop')  )
# song_data = song_data.filter(song_data.Genre != '')
print(song_data.count())
# song_data = song_data.filter(song_data.text != ' ')
# print(song_data.count())
# song_data = song_data.filter(song_data.Genre != ' ')
# print(song_data.count())
song_data.na.drop(subset=["text"])
print(song_data.count())
song_data.show()

10
10
8
8
+--------------------+--------------------+-----+---+
|                 Key|                text|Genre| Id|
+--------------------+--------------------+-----+---+
|10000 maniacs_Mor...|I could feel at t...| Rock|  0|
|10000 maniacs_Bec...|Take me now, baby...| Rock|  1|
|jamiroquai_Rock D...|And it's coming a...|  Pop|  3|
|10000 maniacs_The...|These are. These ...| Rock|  4|
|10000 maniacs_Eve...|Trudging slowly o...| Rock|  6|
|10000 maniacs_Don...|Don't talk, I wil...| Rock|  7|
|black veil brides...|Have we begun to ...| Rock|  8|
|lynyrd skynyrd_I ...|Ain't no need to ...| Rock|  9|
+--------------------+--------------------+-----+---+



In [12]:
trainDataset= song_data.select('Id','text').withColumnRenamed('text','talk_txt').withColumnRenamed('Id','call_transcription_id')
trainDataset.show()
#print(type(song_data))

+---------------------+--------------------+
|call_transcription_id|            talk_txt|
+---------------------+--------------------+
|                    0|I could feel at t...|
|                    1|Take me now, baby...|
|                    3|And it's coming a...|
|                    4|These are. These ...|
|                    6|Trudging slowly o...|
|                    7|Don't talk, I wil...|
|                    8|Have we begun to ...|
|                    9|Ain't no need to ...|
+---------------------+--------------------+



In [13]:
trainDataset= trainDataset.filter(trainDataset.talk_txt != "" )

@udf(StringType())
def collapse_text(val):
    return '. '.join(val)


df2 = trainDataset.groupBy("call_transcription_id").agg(F.collect_list("talk_txt"))
df2.show()

+---------------------+----------------------+
|call_transcription_id|collect_list(talk_txt)|
+---------------------+----------------------+
|                    0|  [I could feel at ...|
|                    1|  [Take me now, bab...|
|                    3|  [And it's coming ...|
|                    4|  [These are. These...|
|                    6|  [Trudging slowly ...|
|                    7|  [Don't talk, I wi...|
|                    8|  [Have we begun to...|
|                    9|  [Ain't no need to...|
+---------------------+----------------------+



In [14]:
df3 = df2.select("call_transcription_id", collapse_text("collect_list(talk_txt)").alias("talk_txt"))
df3.show()
trainDataset=df3
print("dataset loaded")

+---------------------+--------------------+
|call_transcription_id|            talk_txt|
+---------------------+--------------------+
|                    0|I could feel at t...|
|                    1|Take me now, baby...|
|                    3|And it's coming a...|
|                    4|These are. These ...|
|                    6|Trudging slowly o...|
|                    7|Don't talk, I wil...|
|                    8|Have we begun to ...|
|                    9|Ain't no need to ...|
+---------------------+--------------------+

dataset loaded


In [15]:
model_trans_tfId = PipelineModel.load("/content/MyDrive/SparkNLP/tmp/vocab_reduction_1") 
model_trans_tfIdf =  model_trans_tfId.transform(trainDataset)#.persist()
 
model_trans_tfIdf.show()

+---------------------+--------------------+--------------------+
|call_transcription_id|            talk_txt|         cleanTokens|
+---------------------+--------------------+--------------------+
|                    0|I could feel at t...|[could, feel, tim...|
|                    1|Take me now, baby...|[take, baby, hold...|
|                    3|And it's coming a...|[come, baby, yeah...|
|                    4|These are. These ...|[day, youll, reme...|
|                    6|Trudging slowly o...|[trudging, slowly...|
|                    7|Don't talk, I wil...|[dont, talk, list...|
|                    8|Have we begun to ...|[begin, drift, aw...|
|                    9|Ain't no need to ...|[aint, need, worr...|
+---------------------+--------------------+--------------------+



In [16]:
f =open("/content/drive/My Drive/SparkNLP/stopwords_pd.txt", "r")
sw=f.read()

#df = spark.read.csv("/user/eb5377/stopwords_pd.txt")
@udf(StringType())
def sp(x,k):
    import re
    x=x.lower()
    #df = spark.read.csv("/user/eb5377/stopwords_pd.txt")
    k=k.split('\n')
    #stop_wrd = pd.read_csv('/home/EB5377/stopwords_pd.txt')
    for z in k:
        x=re.sub(z,' ',x)
    
    x=re.sub('/s+',' ',x)
    x=x.strip()
    return(x)

@udf(StringType())
def f(x):
    return str(' '.join(x))


@udf(IntegerType())
def sl(x):
    return len(x)

model_trans_tfIdf= model_trans_tfIdf.select('talk_txt','call_transcription_id','cleanTokens',f("cleanTokens").alias("talk_txt2"),sl("talk_txt").alias("str_len"))
model_trans_tfIdf=model_trans_tfIdf.withColumn("sw",lit(sw))

model_trans_tfIdf.cache()
model_trans_tfIdf.show()
trainDataset= model_trans_tfIdf.select('talk_txt','call_transcription_id','cleanTokens',"str_len",sp("talk_txt2","sw").alias("talk_txt3"))

trainDataset.show()


+--------------------+---------------------+--------------------+--------------------+-------+--------------------+
|            talk_txt|call_transcription_id|         cleanTokens|           talk_txt2|str_len|                  sw|
+--------------------+---------------------+--------------------+--------------------+-------+--------------------+
|I could feel at t...|                    0|[could, feel, tim...|could feel time w...|    755|\bapproximately\b...|
|Take me now, baby...|                    1|[take, baby, hold...|take baby hold cl...|   1253|\bapproximately\b...|
|And it's coming a...|                    3|[come, baby, yeah...|come baby yeah co...|   1744|\bapproximately\b...|
|These are. These ...|                    4|[day, youll, reme...|day youll remembe...|    884|\bapproximately\b...|
|Trudging slowly o...|                    6|[trudging, slowly...|trudging slowly w...|    650|\bapproximately\b...|
|Don't talk, I wil...|                    7|[dont, talk, list...|dont ta

In [17]:
trainDataset= trainDataset.filter(trainDataset.talk_txt3 != "" )

In [18]:
#round 2
 
model_trans_tfId = PipelineModel.load("/content/MyDrive/SparkNLP/tmp/vocab_reduction_2") 

model_trans_tfIdf =  model_trans_tfId.transform(trainDataset)#.persist()

model_trans_tfIdf.show()


+--------------------+---------------------+--------------------+-------+--------------------+
|            talk_txt|call_transcription_id|         cleanTokens|str_len|           talk_txt3|
+--------------------+---------------------+--------------------+-------+--------------------+
|I could feel at t...|                    0|[could, feel, tim...|    755|could feel time w...|
|Take me now, baby...|                    1|[take, baby, hold...|   1253|take baby hold cl...|
|And it's coming a...|                    3|[come, baby, yeah...|   1744|come baby yeah co...|
|These are. These ...|                    4|[day, youll, reme...|    884|day youll remembe...|
|Trudging slowly o...|                    6|[trudge, slowly, ...|    650|trudging slowly w...|
|Don't talk, I wil...|                    7|[dont, talk, list...|   1351|dont talk listen ...|
|Have we begun to ...|                    8|[begin, drift, aw...|   1775|begin drift away ...|
|Ain't no need to ...|                    9|[aint,

In [32]:
num_topics = 8
sparsity_df=50
it=80

In [19]:
#Vectorization
from pyspark.ml.feature import CountVectorizer, IDF, CountVectorizerModel
bow_pipeline = CountVectorizerModel.load('/content/drive/MyDrive/SparkNLP/tmp/count_vocab')
bows = bow_pipeline.transform(model_trans_tfIdf)

bows = bows.withColumnRenamed("tf","features") 
bows.show(10) 

+--------------------+---------------------+--------------------+-------+--------------------+--------------------+
|            talk_txt|call_transcription_id|         cleanTokens|str_len|           talk_txt3|            features|
+--------------------+---------------------+--------------------+-------+--------------------+--------------------+
|I could feel at t...|                    0|[could, feel, tim...|    755|could feel time w...|(214,[0,2,3,5,7,1...|
|Take me now, baby...|                    1|[take, baby, hold...|   1253|take baby hold cl...|(214,[0,6,8,10,12...|
|And it's coming a...|                    3|[come, baby, yeah...|   1744|come baby yeah co...|(214,[0,1,2,3,4,5...|
|These are. These ...|                    4|[day, youll, reme...|    884|day youll remembe...|(214,[2,9,11,16,2...|
|Trudging slowly o...|                    6|[trudge, slowly, ...|    650|trudging slowly w...|(214,[5,12,24,73,...|
|Don't talk, I wil...|                    7|[dont, talk, list...|   1351

In [20]:
#LDA (Topic Modeling)
 
from pyspark.ml.clustering import LDA, LDAModel, LocalLDAModel

# lda = LDA(k=num_topics, maxIter=it)

# model = lda.fit(bows)

# model.write().overwrite().save("/content/drive/My Drive/SparkNLP/TM_Targeted.model")
model1 = LocalLDAModel.load("/content/drive/My Drive/SparkNLP/TM_Targeted.model")


In [ ]:
# #extract vocabulary from CountVectorizer

# # ******** SAVE AND LOAD THE MODEL FOR REUSE **************

# vocab = bow_pipeline.vocabulary
 
# topics = model1.describeTopics()   
 
# topics_rdd = topics.rdd
 
# topics_words = topics_rdd.map(lambda row: row['termIndices']).map(lambda idx_list: [vocab[idx] for idx in idx_list]).collect()
# topic_cluster = ''

# for idx, topic in enumerate(topics_words):
#     #print("topic: {}".format(idx))
#     s1 = "topic: {}".format(idx) 
#     #print("*"*25)
#     s2 = '\n' + "*"*25
#     s3 = ''
#     for word in topic:
#         #print(word)
#         s3 =  s3 + '\n' + word
#     s4 =  '\n' + "*"*25  + '\n'
#     #print("*"*25)
#     topic_cluster = topic_cluster + s1 +s2 +s3 +s4


# print(topic_cluster)

In [21]:
lda_model = model1.transform(bows)
lda_model.show()
#lda_model.select('features','topicDistribution').show( truncate = False)

from pyspark.sql.functions import udf, explode
from pyspark.ml.linalg import SparseVector, DenseVector
from pyspark.sql.types import *
import numpy as np

@udf(IntegerType())
def dominant_topics(lst_per):
    lst_per = np.asarray(lst_per.values) #.tolist()
    if float(np.max(lst_per)) == 0.0:
            k=int(9999)
    else:
            k =  int(np.argmax(lst_per))
    return int(k)


@udf(FloatType())
def percent_contrib(lst_per):
    lst_per = np.asarray(lst_per.values) #.tolist()
    if float(np.max(lst_per)) == 0.0:
            k=-9.99
    else:
            k =  float(np.max(lst_per))
    return float(k)


dom_topic_model = lda_model.select('call_transcription_id','talk_txt','str_len','talk_txt3','topicDistribution',dominant_topics("topicDistribution").alias("dom_topic"),percent_contrib("topicDistribution").alias("dom_per"))
dom_topic_model= dom_topic_model.filter(dom_topic_model.dom_topic != 9999 )
dom_topic_model.cache()
dom_topic_model.show()

+--------------------+---------------------+--------------------+-------+--------------------+--------------------+--------------------+
|            talk_txt|call_transcription_id|         cleanTokens|str_len|           talk_txt3|            features|   topicDistribution|
+--------------------+---------------------+--------------------+-------+--------------------+--------------------+--------------------+
|I could feel at t...|                    0|[could, feel, tim...|    755|could feel time w...|(214,[0,2,3,5,7,1...|[0.00245431377749...|
|Take me now, baby...|                    1|[take, baby, hold...|   1253|take baby hold cl...|(214,[0,6,8,10,12...|[0.00194776512034...|
|And it's coming a...|                    3|[come, baby, yeah...|   1744|come baby yeah co...|(214,[0,1,2,3,4,5...|[0.25192971282655...|
|These are. These ...|                    4|[day, youll, reme...|    884|day youll remembe...|(214,[2,9,11,16,2...|[0.00250290313505...|
|Trudging slowly o...|                   

In [ ]:
header='topic|Sample #|Call Transcription ID|Original Text|Model Text|Percent Relevant\n'

phrases_ex=header


for idx in range(num_topics):
    s1 = str("topic: {}".format(idx))
    #s2 = '\n' + "*"*25
    s3 = '|'
    s4=''
    tmp= dom_topic_model.filter(dom_topic_model.dom_topic ==idx) #.filter(dom_topic_model.str_len<=400)
    tmp.cache()
    tmp=tmp.sort(tmp.dom_per.desc())
    txt=tmp.select('talk_txt').collect()    
    txt = [ ele.__getattr__('talk_txt') for ele in txt]
    cid=tmp.select('call_transcription_id').collect()    
    cid = [ ele.__getattr__('call_transcription_id') for ele in cid]
    mtxt=tmp.select('talk_txt3').collect()    
    mtxt = [ ele.__getattr__('talk_txt3') for ele in mtxt]
    per=tmp.select('dom_per').collect()
    per = [ ele.__getattr__('dom_per') for ele in per]
    if len(txt)<40:
        cnt = len(txt)
    else:
        cnt = 40
    for x in range(cnt):
        z=s1+s3+str(x) + s3+ str(cid[x]) +s3+  str(txt[x])+s3+ str(mtxt[x])+s3 +  str("{:.3f}".format(100.0*per[x])) +'%' +'\n'
        s4 =  s4 + z
    #print("*"*25)
    phrases_ex = phrases_ex +s4


print(phrases_ex)

topic|Sample #|Call Transcription ID|Original Text|Model Text|Percent Relevant
topic: 0|0|10000 maniacs_Tolerance|[ Merchant ]. . The still and silence is torn with violence.. A loud breaking sound in the night is made.. Hear it grow, hear it fade.. The sound you're hearing, the sound you're fearing. is the hate that parades up and down our streets, coming within bounds and within reach.. Now, inside the place we hide away, we hear it near and hope it turns away.. Turn away.... There's something seething in the air we're breathing.. We learn slash and burn is the method to use.. Set a flame, burn it new.. We're overpowered.. We kneel, we cower, we cover our heads.. Feel the threat of blows that will come and the damage that will be done in its wake.. Now, inside this place we hide away, we hear it near although it's miles away.. We hear it near and hope it turns away.. Turn away.... This house divided, we live inside it.. Hate's dwelling place is behind our door in fitful nights.. Hear

CREATE TEST DOCUMENT

In [ ]:
model_trans_tfIdf_TEST = model_trans_tfIdf.SELECT()